In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  cross_validate
import math

import sys
from pathlib import Path
# Get the parent directory of the notebook
parent_dir = str(Path().resolve().parent)
# Add it to sys.path
sys.path.append(parent_dir)
from skl_seas_ens import SeasonalClassifier



In [20]:
data = pd.read_csv('data/train.csv', index_col=0)

#add lag data for cloud 
data['cloud_lag_2'] = data['cloud'].shift(2)
data['cloud_step_2'] = data['cloud'].shift(-2)
data['sunshine_lag_1'] = data['sunshine'].shift(1)
data['sunshine_lag_2'] = data['sunshine'].shift(2)
#backfill and forward fill
data.bfill(inplace=True)
data.ffill(inplace=True)
scaler = StandardScaler()
standard_scale_cols = ['pressure', 'mintemp', 'dewpoint',
            'humidity', 'cloud', 'sunshine', 'cloud_lag_2',
            'cloud_step_2', ]
data[standard_scale_cols] = scaler.fit_transform(data[standard_scale_cols])





In [26]:
#Optimize Seasonal Classifier
features = [
            'day',
            'pressure',
            'mintemp', 
            'dewpoint',
            'humidity',
            'cloud', 
            'sunshine', 
            'cloud_lag_2',
            'cloud_step_2',
            'sunshine_lag_1',
            'sunshine_lag_2',
            ]
X = data[features]
y = data['rainfall']

model = SeasonalClassifier(base_model_class= LogisticRegression, 
                           base_model_args=  {'max_iter': 100000, 'tol' : 1e-7, 'random_state' : 0},
                           drop_time_column= True, time_column= 'day', data_is_periodic= True, padding= 135, n_windows= 40, col_names=X.columns)


scoring = {'Accuracy': 'accuracy'}

scores = cross_validate(model, X, y, cv=5,scoring=scoring)
print("Accuracy Seasonal Classifier: ", scores['test_Accuracy'].mean()*100)






/home/max/anaconda3/envs/zalando/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/max/anaconda3/envs/zalando/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/max/anaconda3/envs/zalando/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/max/anaconda3/envs/

Accuracy Seasonal Classifier:  86.71232876712327


/home/max/anaconda3/envs/zalando/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/max/anaconda3/envs/zalando/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/home/max/anaconda3/envs/zalando/lib/python3.12/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [27]:
#remove day from features
features.remove('day')
X = data[features]
y = data['rainfall']

model = LogisticRegression(max_iter=100000, tol =1e-7,random_state=0)
scoring = {'Accuracy': 'accuracy'}
scores = cross_validate(model, X, y, cv=5,scoring=scoring)
print("Accuracy Logistic Regression ", scores['test_Accuracy'].mean()*100)


Accuracy Logistic Regression  86.57534246575341
